## Python Tutorial

This tutorial is meant to show you how to work with the `symd`. This will not focus on running simulations -- see the other examples for that. Let's import the package:

In [ ]:
import symd

### A group

All 2D and 3D groups that tile periodically can be loaded with `symd` (see custom group example to make more). Let's load a 2D group using its Hall Number (1-17 in 2D).}

In [ ]:
my_group = symd.load_group(11, dim=2)
print(my_group)

You can see the group is a data structure consisting of the Bravais lattice, the general positions (group elements represented as affine matrices), special positions (itself a group), and an asymmetric unit specified as an inequality. The group in this format cannot be used for many calculations, so we'll need to convert these strings into python objects. 

#### Group Elements

Let's get access to the group elements and apply them to some points. We'll access them using the {obj}`symd.str2mat` function. These are affine matrices, so they are shape $D+1\times D+1$, where $D$ is the number of dimensions. Thus, we need to add work in [homogeneous coordinates](https://en.wikipedia.org/wiki/Homogeneous_coordinates), which are $(x,y,1)$. 

We'll just apply the group elements to these homogeneous coordinates and plot the results. Remember we wrap the coordinates here, as done in the engine and paper.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 8
D = 2
some_points = np.random.rand(N, D)
homo_points = np.concatenate((some_points, np.ones((N, 1))), axis=-1)

for s in my_group.genpos:
    g = symd.str2mat(s)
    xp = homo_points @ g
    xp = np.fmod(xp, 1.0)
    plt.plot(xp[:, 0], xp[:, 1], "o")

Now let's try applying the special positions, also known as Wyckoff sites.

In [ ]:
for sp in my_group.specpos:
    for s in sp.genpos:
        g = symd.str2mat(s)
        xp = homo_points @ g
        xp = np.fmod(xp, 1.0)
        plt.plot(xp[:, 0], xp[:, 1], "o")

You can see the two sub groups: points on the line and the origin.

#### Asymmetric unit

As you add more point, you'll notice that there is overlap:

In [ ]:
N = 8000
D = 2
some_points = np.random.rand(N, D)
homo_points = np.concatenate((some_points, np.ones((N, 1))), axis=-1)

for s in my_group.genpos:
    g = symd.str2mat(s)
    xp = homo_points @ g
    xp = np.fmod(xp, 1.0)
    plt.plot(xp[:, 0], xp[:, 1], ".")

This is because our randomly generated points do not all lie in the asymmetric unit. We can load the asymmetric unit test function and use it to filter our points

In [ ]:
in_unit = symd.asymm_constraints(my_group.asymm_unit)
print(in_unit(0.2, -0.3))
print(in_unit(0.2, 0.3))

In [ ]:
# filter out those not in asmmetric unit
some_points = some_points[[in_unit(*p) for p in some_points]]
homo_points = np.concatenate((some_points, np.ones((some_points.shape[0], 1))), axis=-1)

for s in my_group.genpos:
    g = symd.str2mat(s)
    xp = homo_points @ g
    xp = np.fmod(xp, 1.0)
    plt.plot(xp[:, 0], xp[:, 1], ".")

### Bravais Lattice

To project lattice vectors into the correct Bravais lattice, we can use the {obj}`symd.project_cell` function:

In [ ]:
lattice_vectors = np.random.uniform(size=(2, 2))
blattice_vectors = symd.project_cell(lattice_vectors, my_group.lattice)
print("This lattice is", my_group.lattice)
print(blattice_vectors)

There are a few other convenience functions, like counting the number of particles in a cell if you duplicate for all sites, and getting the cell volume

In [ ]:
print("The volume of my cell is", symd.cell_volume(blattice_vectors))
print(
    "If I have 8 particles in asymmetric unit, there will be",
    symd.cell_nparticles(my_group, 8),
    "in the unit cell",
)
print(
    "If I want to have a number density of 0.6 with 8 particles,\nmy cell should be",
    symd.get_cell(0.6, my_group, dim=2, n=8),
)